In [167]:
Code Example: https://github.com/adventuresinML/adventures-in-ml-code/blob/master/r_learning_tensorflow.py

In [11]:
import numpy as np
import tensorflow as tf
import gym
import matplotlib.pylab as plt
import random
import math

MAX_EPSILON = 1
MIN_EPSILON = 0.01
LAMBDA = 0.0001
GAMMA = 0.99
BATCH_SIZE = 50

In [12]:
#def takeaction(env):
    #a = env.action_space.sample()
    #print("This is the next action sample: {} {} {} {}".format(a[0],a[1],a[2],a[3]))
    #return a
#    return[0.1,0,0,0]

In [13]:
#env_name = "FetchPickAndPlace-v1"
#env = gym.make(env_name)

#env.reset()


In [14]:
#env.render()
#env.step(takeaction(env))

In [15]:


class Model:
    def __init__(self, num_states, num_actions, batch_size):
        self._num_states = num_states
        self._num_actions = num_actions
        self._batch_size = batch_size
        # define the placeholders
        self._states = None
        self._actions = None
        # the output operations
        self._logits = None
        self._optimizer = None
        self._var_init = None
        # now setup the model
        self._define_model()

    def _define_model(self):
        self._states = tf.placeholder(shape=[None, self._num_states], dtype=tf.float32)
        self._q_s_a = tf.placeholder(shape=[None, self._num_actions], dtype=tf.float32)
        # create a couple of fully connected hidden layers
        fc1 = tf.layers.dense(self._states, 50, activation=tf.nn.relu)
        fc2 = tf.layers.dense(fc1, 50, activation=tf.nn.relu)
        self._logits = tf.layers.dense(fc2, self._num_actions)
        loss = tf.losses.mean_squared_error(self._q_s_a, self._logits)
        self._optimizer = tf.train.AdamOptimizer().minimize(loss)
        self._var_init = tf.global_variables_initializer()

    def predict_one(self, state, sess):
        return sess.run(self._logits, feed_dict={self._states:
                                                     state.reshape(1, self.num_states)})

    def predict_batch(self, states, sess):
        return sess.run(self._logits, feed_dict={self._states: states})

    def train_batch(self, sess, x_batch, y_batch):
        sess.run(self._optimizer, feed_dict={self._states: x_batch, self._q_s_a: y_batch})

    @property
    def num_states(self):
        return self._num_states

    @property
    def num_actions(self):
        return self._num_actions

    @property
    def batch_size(self):
        return self._batch_size

    @property
    def var_init(self):
        return self._var_init


In [16]:

class Memory:
    def __init__(self, max_memory):
        self._max_memory = max_memory
        self._samples = []

    def add_sample(self, sample):
        self._samples.append(sample)
        if len(self._samples) > self._max_memory:
            self._samples.pop(0)

    def sample(self, no_samples):
        if no_samples > len(self._samples):
            return random.sample(self._samples, len(self._samples))
        else:
            return random.sample(self._samples, no_samples)
        

In [57]:

class GameRunner:
    def __init__(self, sess, model, env, memory, max_eps, min_eps,
                 decay, render=True):
        self._sess = sess
        self._env = env
        self._model = model
        self._memory = memory
        self._render = render
        self._max_eps = max_eps
        self._min_eps = min_eps
        self._decay = decay
        self._eps = self._max_eps
        self._steps = 0
        self._reward_store = []
        

    def run(self):
        state = self._env.reset()["observation"]
        tot_reward = 0
        while True:
            if self._render:
                self._env.render()

            action = self._choose_action(state)
            temp_state, reward, done, info = self._env.step(action)
            
            # her
            temp_state["desired_goal"] = temp_state["achieved_goal"]
            done = True
            #print ("Des: {}".format(temp_state["desired_goal"]))
            #print ("Ach: {}".format(temp_state["achieved_goal"]))
            
            
            print("State {}".format(temp_state))
            next_state = temp_state["observation"]
            
            
            #if done:
            #    if reward >= 0:
            #       print("YEAH")
            #       reward += 1000

            # is the game complete? If so, set the next state to
            # None for storage sake
            if done:
                next_state = None

            self._memory.add_sample((state, action, reward, next_state))
            self._replay()

            # exponentially decay the eps value
            self._steps += 1
            self._eps = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) \
                                      * math.exp(-LAMBDA * self._steps)

            # move the agent to the next state and accumulate the reward
            state = next_state
            tot_reward += reward

            # if the game is done, break the loop
            if done:
                self._reward_store.append(tot_reward)
                
                break

        print("Step {}, Total reward: {}, Eps: {}".format(self._steps, tot_reward, self._eps))

    def _choose_action(self, state):
        if random.random() < self._eps:
            return random.randint(0, self._model.num_actions - 1)  #self._env.action_space.sample()
        else:
            return np.argmax(self._model.predict_one(state, self._sess))

    def _replay(self):
        batch = self._memory.sample(self._model.batch_size)
        states = np.array([val[0] for val in batch])
        next_states = np.array([(np.zeros(self._model.num_states)
                                 if val[3] is None else val[3]) for val in batch])
        # predict Q(s,a) given the batch of states
        q_s_a = self._model.predict_batch(states, self._sess)
        # predict Q(s',a') - so that we can do gamma * max(Q(s'a')) below
        q_s_a_d = self._model.predict_batch(next_states, self._sess)
        # setup training arrays
        x = np.zeros((len(batch), self._model.num_states))
        y = np.zeros((len(batch), self._model.num_actions))
        for i, b in enumerate(batch):
            state, action, reward, next_state = b[0], b[1], b[2], b[3]
            # get the current q values for all actions in state
            current_q = q_s_a[i]
            # update the q value for action
            if next_state is None:
                # in this case, the game completed after action, so there is no max Q(s',a')
                # prediction possible
                current_q[action] = reward
            else:
                current_q[action] = reward + GAMMA * np.amax(q_s_a_d[i])
            x[i] = state
            y[i] = current_q
        self._model.train_batch(self._sess, x, y)

    @property
    def reward_store(self):
        return self._reward_store


    

In [58]:
if __name__ == "__main__":
    env_name = 'FetchPickAndPlace-v1'
    env = gym.make(env_name)

    num_states = env.observation_space["observation"].shape[0]
    print(num_states)
    num_actions = env.action_space.shape[0]
    print(num_actions)
    
    model = Model(num_states, num_actions, BATCH_SIZE)
    mem = Memory(50000)

    with tf.Session() as sess:
        sess.run(model.var_init)
        gr = GameRunner(sess, model, env, mem, MAX_EPSILON, MIN_EPSILON,
                        LAMBDA)
        num_episodes = 500
        cnt = 0
        while cnt < num_episodes:
            if cnt % 10 == 0:
                print('Episode {} of {}'.format(cnt+1, num_episodes))
            gr.run()
            cnt += 1
        plt.plot(gr.reward_store)
        plt.show()
        plt.close("all")


25
4
Episode 1 of 500
Creating window glfw
State {'achieved_goal': array([1.23566991, 0.79149706, 0.42473605]), 'desired_goal': array([1.23566991, 0.79149706, 0.42473605]), 'observation': array([ 1.36945404e+00,  7.78820325e-01,  5.64114120e-01,  1.23566991e+00,
        7.91497062e-01,  4.24736048e-01, -1.33784128e-01,  1.26767376e-02,
       -1.39378071e-01,  3.97852140e-02,  4.16259342e-02, -3.85214084e-07,
        5.92637053e-07,  1.12208536e-13, -2.49958924e-02, -2.62562578e-02,
       -2.64169060e-02,  1.87589293e-07, -2.88598912e-07, -1.50758860e-18,
        2.49958852e-02,  2.62562532e-02,  2.64435715e-02,  6.99533302e-02,
        7.08374623e-02])}
State {'achieved_goal': array([1.2356699 , 0.79149706, 0.42475675]), 'desired_goal': array([1.2356699 , 0.79149706, 0.42475675]), 'observation': array([ 1.40038018e+00,  8.12697465e-01,  5.97742402e-01,  1.23566990e+00,
        7.91497058e-01,  4.24756751e-01, -1.64710274e-01, -2.12004069e-02,
       -1.72985651e-01,  5.14205340e-02, 

State {'achieved_goal': array([1.2356698 , 0.79149699, 0.42478448]), 'desired_goal': array([1.2356698 , 0.79149699, 0.42478448]), 'observation': array([ 1.41848121e+00,  1.04493740e+00,  7.81122807e-01,  1.23566980e+00,
        7.91496988e-01,  4.24784482e-01, -1.82811414e-01, -2.53440411e-01,
       -3.56338325e-01,  5.00469908e-02,  5.00427364e-02, -3.85214084e-07,
        5.92637053e-07,  1.12208536e-13,  5.73169258e-03, -1.05293628e-02,
       -6.25901637e-03,  1.85977012e-07, -2.86118480e-07,  3.41864130e-18,
       -5.73169970e-03,  1.05293582e-02,  6.25902002e-03, -1.80752830e-06,
       -3.43525028e-08])}
State {'achieved_goal': array([1.23566979, 0.79149698, 0.42478449]), 'desired_goal': array([1.23566979, 0.79149698, 0.42478449]), 'observation': array([ 1.40426854e+00,  1.04085955e+00,  7.76073975e-01,  1.23566979e+00,
        7.91496984e-01,  4.24784485e-01, -1.68598754e-01, -2.49362567e-01,
       -3.51289489e-01,  4.99314210e-02,  4.96054649e-02, -3.85214084e-07,
        5

State {'achieved_goal': array([1.2356697 , 0.79149692, 0.42478449]), 'desired_goal': array([1.2356697 , 0.79149692, 0.42478449]), 'observation': array([ 1.33224792e+00,  1.14646031e+00,  8.30395057e-01,  1.23566970e+00,
        7.91496923e-01,  4.24784489e-01, -9.65782227e-02, -3.54963384e-01,
       -4.05610568e-01,  5.00480070e-02,  5.00416351e-02, -3.85214084e-07,
        5.92637053e-07,  1.12208536e-13,  3.06743163e-03, -5.72628992e-03,
       -2.85920789e-03,  1.85976796e-07, -2.86118148e-07, -2.61737623e-18,
       -3.06743876e-03,  5.72628529e-03,  2.85920789e-03, -3.00351960e-06,
        1.41399551e-06])}
State {'achieved_goal': array([1.23566969, 0.79149692, 0.42478449]), 'desired_goal': array([1.23566969, 0.79149692, 0.42478449]), 'observation': array([ 1.32764875e+00,  1.15424831e+00,  8.34039783e-01,  1.23566969e+00,
        7.91496919e-01,  4.24784489e-01, -9.19790614e-02, -3.62751387e-01,
       -4.09255294e-01,  5.00465287e-02,  5.00426114e-02, -3.85214084e-07,
        5

State {'achieved_goal': array([1.23566959, 0.79149685, 0.42478449]), 'desired_goal': array([1.23566959, 0.79149685, 0.42478449]), 'observation': array([ 1.27875935e+00,  1.20447990e+00,  8.51340406e-01,  1.23566959e+00,
        7.91496854e-01,  4.24784489e-01, -4.30897625e-02, -4.12983050e-01,
       -4.26555917e-01,  5.00459540e-02,  5.00431576e-02, -3.85214084e-07,
        5.92637053e-07,  1.12208536e-13,  1.43747344e-03, -2.67997150e-03,
       -1.06886736e-03,  1.85976796e-07, -2.86118148e-07,  1.26832039e-18,
       -1.43748056e-03,  2.67996687e-03,  1.06886736e-03, -1.29552887e-06,
        1.27970425e-06])}
State {'achieved_goal': array([1.23566958, 0.79149685, 0.42478449]), 'desired_goal': array([1.23566958, 0.79149685, 0.42478449]), 'observation': array([ 1.26638966e+00,  1.19490526e+00,  8.43542713e-01,  1.23566958e+00,
        7.91496849e-01,  4.24784489e-01, -3.07200754e-02, -4.03408415e-01,
       -4.18758224e-01,  4.97716669e-02,  4.97234845e-02, -3.85214084e-07,
        5

State {'achieved_goal': array([1.46840356, 0.63250729, 0.4247843 ]), 'desired_goal': array([1.46840356, 0.63250729, 0.4247843 ]), 'observation': array([ 1.45644973e+00,  8.77671375e-01,  6.60316392e-01,  1.46840356e+00,
        6.32507293e-01,  4.24784302e-01,  1.19538316e-02, -2.45164081e-01,
       -2.35532089e-01,  5.00414807e-02,  5.03366052e-02, -3.85214084e-07,
        5.92637053e-07,  1.12208536e-13, -1.99289036e-02, -2.53578893e-02,
       -2.48121043e-02,  1.85982836e-07, -2.86127440e-07, -1.04198276e-18,
        1.99288965e-02,  2.53578846e-02,  2.48122065e-02,  1.07490840e-05,
       -8.21264308e-04])}
State {'achieved_goal': array([1.46840355, 0.63250729, 0.42478438]), 'desired_goal': array([1.46840355, 0.63250729, 0.42478438]), 'observation': array([ 1.46954485e+00,  9.07376450e-01,  6.86566586e-01,  1.46840355e+00,
        6.32507289e-01,  4.24784382e-01, -1.14129219e-03, -2.74869161e-01,
       -2.61782204e-01,  5.00471707e-02,  5.00606277e-02, -3.85214084e-07,
        5

...

State {'achieved_goal': array([1.41996357, 0.83171957, 0.4247843 ]), 'desired_goal': array([1.41996357, 0.83171957, 0.4247843 ]), 'observation': array([ 1.44078353e+00,  1.01426639e+00,  7.63640518e-01,  1.41996357e+00,
        8.31719569e-01,  4.24784302e-01, -2.08199546e-02, -1.82546825e-01,
       -3.38856215e-01,  5.00456973e-02,  5.00434093e-02, -3.85214084e-07,
        5.92637053e-07,  1.12208536e-13,  6.28167071e-03, -1.19139606e-02,
       -7.36038033e-03,  1.85982836e-07, -2.86127440e-07,  1.89472169e-19,
       -6.28167783e-03,  1.19139560e-02,  7.36048258e-03, -3.88781227e-07,
        3.88019364e-07])}
State {'achieved_goal': array([1.41996357, 0.83171956, 0.42478438]), 'desired_goal': array([1.41996357, 0.83171956, 0.42478438]), 'observation': array([ 1.43155270e+00,  1.03077274e+00,  7.73621868e-01,  1.41996357e+00,
        8.31719564e-01,  4.24784382e-01, -1.15891334e-02, -1.99053180e-01,
       -3.48837486e-01,  5.00454821e-02,  5.00436243e-02, -3.85214084e-07,
        5

State {'achieved_goal': array([1.41996347, 0.8317195 , 0.42478449]), 'desired_goal': array([1.41996347, 0.8317195 , 0.42478449]), 'observation': array([ 1.33119019e+00,  1.14437733e+00,  8.27677084e-01,  1.41996347e+00,
        8.31719499e-01,  4.24784489e-01,  8.87732790e-02, -3.12657832e-01,
       -4.02892595e-01,  5.00524260e-02,  5.00466504e-02, -3.85214084e-07,
        5.92637053e-07,  1.12208536e-13,  2.38082242e-03, -6.10067901e-03,
       -3.32535791e-03,  1.85976798e-07, -2.86118150e-07,  2.35715776e-18,
       -2.38082954e-03,  6.10067438e-03,  3.32535793e-03, -1.07248426e-05,
       -1.88409539e-05])}
State {'achieved_goal': array([1.41996346, 0.83171949, 0.42478449]), 'desired_goal': array([1.41996346, 0.83171949, 0.42478449]), 'observation': array([ 1.32723311e+00,  1.15266370e+00,  8.32015864e-01,  1.41996346e+00,
        8.31719495e-01,  4.24784489e-01,  9.27303472e-02, -3.20944203e-01,
       -4.07231375e-01,  5.00479983e-02,  5.00417353e-02, -3.85214084e-07,
        5

State {'achieved_goal': array([1.41996336, 0.83171943, 0.42478449]), 'desired_goal': array([1.41996336, 0.83171943, 0.42478449]), 'observation': array([ 1.28259259e+00,  1.20132711e+00,  8.50689889e-01,  1.41996336e+00,
        8.31719430e-01,  4.24784489e-01,  1.37370775e-01, -3.69607683e-01,
       -4.25905400e-01,  5.00455446e-02,  5.00435620e-02, -3.85214084e-07,
        5.92637053e-07,  1.12208536e-13,  1.67153846e-03, -2.76347616e-03,
       -1.02984764e-03,  1.85976796e-07, -2.86118148e-07, -5.64297488e-19,
       -1.67154559e-03,  2.76347153e-03,  1.02984764e-03, -9.35836005e-07,
        9.34844754e-07])}
State {'achieved_goal': array([1.41996335, 0.83171943, 0.42478449]), 'desired_goal': array([1.41996335, 0.83171943, 0.42478449]), 'observation': array([ 1.27001575e+00,  1.19176345e+00,  8.42802165e-01,  1.41996335e+00,
        8.31719425e-01,  4.24784489e-01,  1.49947608e-01, -3.60044028e-01,
       -4.18017675e-01,  4.97736238e-02,  4.97206405e-02, -3.85214084e-07,
        5

State {'achieved_goal': array([1.20270869, 0.88246571, 0.42478149]), 'desired_goal': array([1.20270869, 0.88246571, 0.42478149]), 'observation': array([ 1.46832755e+00,  9.32750999e-01,  7.07434129e-01,  1.20270869e+00,
        8.82465709e-01,  4.24781489e-01, -2.65618860e-01, -5.02852896e-02,
       -2.82652640e-01,  5.00482879e-02,  5.00408672e-02, -3.85214084e-07,
        5.92637053e-07,  1.12208536e-13,  1.77539195e-03, -1.67379564e-02,
       -1.27377041e-02,  1.86074041e-07, -2.86267755e-07, -2.56293013e-18,
       -1.77539908e-03,  1.67379517e-02,  1.27393480e-02, -2.86398628e-06,
        2.71949883e-06])}


KeyboardInterrupt: 